# DuckDB unleashed: Scratching the surface a little deeper

In [4]:
import duckdb
import pandas as pd
pd.set_option('display.max_columns', 50)

## DuckDB connections

In [5]:
c =  duckdb.connect()
c.execute("select * from '../data/transactions.parquet' limit 5").df()

,transaction_time,id,shop
0,2022-02-11 00:28:50,480571,46004
1,2022-03-20 10:41:34,113107,746
2,2022-08-17 03:59:32,668523,49808
3,2022-09-08 06:24:04,731871,30563
4,2022-12-13 19:01:15,415373,18267


## Querying online sources

In [6]:
%%time
duckdb.sql(
    """
    select
    *
    from read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-09.parquet') as yc
    limit 5
    """
).df()

CPU times: user 1.11 s, sys: 16 ms, total: 1.13 s
Wall time: 1.19 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2024-09-01 00:05:51,2024-09-01 00:45:03,1,9.80,1,N,138,48,1,47.8,10.25,0.5,13.30,6.94,1.0,79.79,2.5,1.75
1,1,2024-09-01 00:59:35,2024-09-01 01:03:43,1,0.50,1,N,140,141,1,5.1,3.50,0.5,3.00,0.00,1.0,13.10,2.5,0.00
2,2,2024-09-01 00:25:00,2024-09-01 00:34:37,2,2.29,1,N,238,152,2,13.5,1.00,0.5,0.00,0.00,1.0,16.00,0.0,0.00
3,2,2024-09-01 00:31:00,2024-09-01 00:46:52,1,5.20,1,N,93,130,1,24.7,1.00,0.5,4.55,0.00,1.0,31.75,0.0,0.00
4,2,2024-09-01 00:11:57,2024-09-01 00:30:41,2,2.26,1,N,79,231,1,17.0,1.00,0.5,4.40,0.00,1.0,26.40,2.5,0.00


In [7]:
%%time
duckdb.sql(
    """
    select
    dayofweek(tpep_pickup_datetime) as dow
    , round(sum(yc.tip_amount)/sum(yc.total_amount - yc.tip_amount), 3) as tip_pct
    , round(mean(case when yc.tip_amount/(yc.total_amount - yc.tip_amount) < 0.1 then 1 else 0 end), 3) as share_under_10
    from read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-09.parquet') as yc
    group by dayofweek(tpep_pickup_datetime)
    """
)

CPU times: user 8.93 ms, sys: 1.56 ms, total: 10.5 ms
Wall time: 153 ms


┌───────┬─────────┬────────────────┐
│  dow  │ tip_pct │ share_under_10 │
│ int64 │ double  │     double     │
├───────┼─────────┼────────────────┤
│     0 │   0.125 │           0.43 │
│     1 │   0.135 │          0.375 │
│     2 │   0.135 │           0.37 │
│     3 │   0.137 │          0.362 │
│     4 │   0.135 │          0.371 │
│     5 │    0.13 │            0.4 │
│     6 │   0.122 │          0.425 │
└───────┴─────────┴────────────────┘

All 2024 data have also been uploaded in https://oss.buz.dev/ - courtesy of https://bsky.app/profile/jakthom.bsky.social

In [8]:
c =  duckdb.connect()
c.execute("ATTACH 'https://hive.buz.dev/nyc_taxi' AS nyc_taxi;")
c.execute("SHOW ALL TABLES;").df()
c.execute("DESC nyc_taxi.yellow_trips").df()

,column_name,column_type,null,key,default,extra
0,VendorID,INTEGER,YES,None,None,None
1,tpep_pickup_datetime,TIMESTAMP,YES,None,None,None
2,tpep_dropoff_datetime,TIMESTAMP,YES,None,None,None
3,passenger_count,BIGINT,YES,None,None,None
4,trip_distance,DOUBLE,YES,None,None,None
5,RatecodeID,BIGINT,YES,None,None,None
6,store_and_fwd_flag,VARCHAR,YES,None,None,None
7,PULocationID,INTEGER,YES,None,None,None
8,DOLocationID,INTEGER,YES,None,None,None
9,payment_type,BIGINT,YES,None,None,None


### Foursquare place data

Again thanks to https://bsky.app/profile/jakthom.bsky.social

In [9]:
c =  duckdb.connect()
c.execute("ATTACH 'https://hive.buz.dev/foursquare' AS foursquare;")
c.execute("USE foursquare;")
c.execute("SHOW TABLES").df()

,name
0,categories
1,places


In [11]:
%time
c.execute("select * from places where country='DK' limit 7").df()

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.34 µs


,fsq_place_id,name,latitude,longitude,address,locality,region,postcode,admin_region,post_town,po_box,country,date_created,date_refreshed,date_closed,tel,website,email,facebook_id,instagram,twitter,fsq_category_ids,fsq_category_labels,dt
0,1ca1519f47f14c4f0f7910a9,Freepete,55.683642,12.532740,"Rolfsvej 33, 4. sal. tv",Frederiksberg,Hovedstaden,2000,None,None,None,DK,2015-11-26,2023-12-06,None,23 29 00 00,None,peter@hingebjerg.dk,NaN,None,None,[63be6904847c3692a84b9b72],[Business and Professional Services > Media Ag...,2024-11-19
1,bfeb38ad0a564eb2aa55445b,Salon Cherie,55.637152,12.086013,Køgevej 44,Roskilde,Sjælland,4000,None,None,None,DK,2013-10-24,2023-12-06,None,46 35 28 88,None,None,NaN,None,None,[4bf58dd8d48988d110951735],[Business and Professional Services > Health a...,2024-11-19
2,4ef8724e0e61a0846e3f4347,Hedelykke,56.442661,8.957332,Herningvej 66,Skive,None,7800,None,None,None,DK,2011-12-26,2020-11-23,None,None,None,None,NaN,None,None,[4bf58dd8d48988d15b941735],[Landmarks and Outdoors > Farm],2024-11-19
3,fb77559581614f90ed4dbabb,Vintur.dk ApS,55.531863,12.204529,Bæktoften 10,Solrød Strand,Sjælland,2680,None,None,None,DK,2017-09-01,2023-08-17,None,22 38 78 08,None,None,NaN,None,None,[4f04b08c2fb6e1c99f3db0bd],[Travel and Transportation > Travel Agency],2024-11-19
4,5a3d1dcc22cb41af42059da2,Danofix-Tape ApS,55.646326,12.124600,Betonvej 13,Roskilde,Sjælland,4000,None,None,None,DK,2014-06-01,2023-08-19,None,46 75 43 00,http://www.danofix.dk,danofix@danofix.dk,NaN,None,None,NaN,NaN,2024-11-19
5,81becd8d369946547d3feb34,Grøndahl Trafikteknik,55.641527,12.594715,Stenlandsvej 25,København,Hovedstaden,2300,None,None,None,DK,2015-05-06,2024-05-31,None,32 55 05 01,http://www.atkins.dk,knudgm@image.dk,NaN,None,None,[4d4b7105d754a06375d81259],[Business and Professional Services],2024-11-19
6,2359dbb22c134a5b12cbdde1,Egel-Data,54.948393,10.727639,"Peløkkevej 34, Peløkke",Rudkøbing,Syddanmark,5900,None,None,None,DK,2014-09-08,2023-12-06,None,62 51 14 50,None,None,NaN,None,None,[63be6904847c3692a84b9b3e],[Business and Professional Services > Financia...,2024-11-19
